In [1]:
import pandas as pd
import numpy as np

# 0. Предообработка данных

In [2]:
# загрузил файл, посмотрел на шапку и данные
df=pd.read_csv("D:\\SWORK\\Тестовые задания\\Uma.Tech\\upd_task_rpl.csv\\upd_task_rpl.csv")
df.head()

,created,title,name,id
0,2018-09-02 19:12:22.144047,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,fa18fdf154aa0defaf386d52f770ff52
1,2018-08-30 13:16:12.620303,Ростов - Рубин. Российская Премьер-лига. 6 тур,Трансляция РПЛ,0ae165a76e8730674f61bb91c0aa108b
2,2018-09-02 19:27:45.044487,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88b3dc521850617dc465eb7936282357
3,2018-09-01 14:50:46.626348,ЦСКА - Урал. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88dd235c1eeb4be7b39637730e54c121
4,2018-09-01 18:42:08.190768,Краснодар - Локомотив. Российская Премьер-лига...,Трансляция РПЛ,1d6384cc9a6917f2dd1b7df7a9681eb2


In [3]:
# сделал копию, оставив исходную загрузку не тронутой, на случай если нужны будут исходные данные в нетронутом виде
df2=df.copy(deep=True)

In [4]:
# в данном блоке извлек дату из колонки created, распарсил значение даты на год, месяц, день, преобразовал форматы данных
df2['date']=df['created'].str.split(' ').str.get(0)

df2['year']=df2['date'].str.split('-').str.get(0)
df2['month']=df2['date'].str.split('-').str.get(1)
df2['day']=df2['date'].str.split('-').str.get(2)

df2['new_season']=df2['date']
df2['date']=pd.to_datetime(pd.Series(df2['date']))
df2['year']=df2['year'].astype(str).astype(int)
df2['day']=df2['day'].astype(str).astype(int)

df2['Marker']= 1

In [5]:
#  конвертировал значенни месяца в читаемый формат
df2.loc[df2['month'] == '01', 'month'] = '01.Январь'
df2.loc[df2['month'] == '02', 'month'] = '02.Февраль'
df2.loc[df2['month'] == '03', 'month'] = '03.Март'
df2.loc[df2['month'] == '04', 'month'] = '04.Апрель'
df2.loc[df2['month'] == '05', 'month'] = '05.Май'
df2.loc[df2['month'] == '06', 'month'] = '06.Июнь'
df2.loc[df2['month'] == '07', 'month'] = '07.Июль'
df2.loc[df2['month'] == '08', 'month'] = '08.Август'
df2.loc[df2['month'] == '09', 'month'] = '09.Сентябрь'
df2.loc[df2['month'] == '10', 'month'] = '10.Октябрь'
df2.loc[df2['month'] == '11', 'month'] = '11.Ноябрь'
df2.loc[df2['month'] == '12', 'month'] = '12.Декабрь'

In [6]:
# место ввода даты обозначающей начало нового сезона
input_date = '2019-06-30'

In [7]:
# в данном блоке делаю разметку данных по двум параметрам Старый/Новый сезон, Подписка/Трансляция
df2.loc[df2['new_season'] > input_date, 'new_season'] = 'Новый_сезон_РПЛ'
df2.loc[df2['new_season'] <= input_date, 'new_season'] = 'Старый_сезон_РПЛ'

# в качестве патерна разделения на подписку и трансляции отобрал все строки из колонки 'name'
# которые начинались на слово МАТЧ и обозначил их как "Подписка". Все остальные "Трансляция"
df2['SeasP-Transl']=df2['name']
df2['SeasP-Transl_1']=df2['SeasP-Transl'].str.split(' ').str.get(0)
df2.loc[df2['SeasP-Transl_1'] == 'МАТЧ', 'SeasP-Transl'] = 'Подписка'
df2.loc[df2['SeasP-Transl_1'] != 'МАТЧ', 'SeasP-Transl'] = 'Трансляция'
del df2['SeasP-Transl_1']

=========================================================================================================================

# Вопрос №1. Кол-во уникальных юзеров/покупок в каждом месяце.

In [8]:
# подсчет и вывод кол-ва продаж по месяцам
count_sales = df2.groupby(['year', 'month']).size().reset_index(name='sales')
count_sales

,year,month,sales
0,2018,08.Август,44436
1,2018,09.Сентябрь,43914
2,2018,10.Октябрь,34764
3,2018,11.Ноябрь,37895
4,2018,12.Декабрь,18287
5,2019,01.Январь,238
6,2019,02.Февраль,261
7,2019,03.Март,45293
8,2019,04.Апрель,99917
9,2019,05.Май,43067


In [9]:
# подсчет и вывод кол-ва уникальных клиентов по месяцам (уникальных id)
count_clients = df2.copy(deep=True)
count_clients=count_clients.drop_duplicates(subset=['id', 'year', 'month'], keep='first')
count_clients_2 = count_clients.groupby(['year', 'month']).size().reset_index(name='qnt_client')
count_clients_2

,year,month,qnt_client
0,2018,08.Август,33904
1,2018,09.Сентябрь,32654
2,2018,10.Октябрь,26614
3,2018,11.Ноябрь,29290
4,2018,12.Декабрь,15617
5,2019,01.Январь,208
6,2019,02.Февраль,242
7,2019,03.Март,36258
8,2019,04.Апрель,69706
9,2019,05.Май,31621


In [10]:
# сформировал сводную таблицу по клиентам / сезонам / подпискам.
# далее таблица по таблице будут делаться расчется и проставляться разметка для ответа на вопросы 2-5.
table = pd.pivot_table(df2, values='Marker', index=['id'], columns=['new_season','SeasP-Transl'], aggfunc=np.sum)
table_2 = table.copy(deep=True)
table_2['Marker'] = 1

=========================================================================================================================

 Вопрос №1. Кол-во уникальных юзеров/покупок в каждом месяце.

 Краткое резюму по вопросу №1:
 - Динамика кол-ва продаж показывает что пик пришелся на Апрель 2018
 - "Провальными оказались Январь 2019, Февраль 2019, Июнь 2019"
 - Малые значения по Августу 2019 в расчет брать нельзя т.к. по данному месяцу данные представлены только по 1 августа
 - Анализ статистики кол-ва уникальных клиентов в разрезе месяцов так же подтверждает сложившуюся ситуацию по пикам и провалам продаж

=========================================================================================================================

# Вопрос №2. Кто покупает подписки в новом сезоне — новые люди, или у них были и раньше подписки? 
# А может, они устали покупать трансляции и перешли на подписку?

In [11]:
# добавляю колонки с маркерами
table_2.loc[table_2['Новый_сезон_РПЛ']['Подписка'] > 0, 'Н_РПЛ_П_С1'] = 2000
table_2.loc[table_2['Новый_сезон_РПЛ']['Трансляция'] > 0, 'Н_РПЛ_Т_С1'] = 500
table_2.loc[table_2['Старый_сезон_РПЛ']['Подписка'] > 0, 'С_РПЛ_П_С1'] = 1000
table_2.loc[table_2['Старый_сезон_РПЛ']['Трансляция'] > 0, 'С_РПЛ_Т_С1'] = 100
table_2= table_2.fillna(0)

In [12]:
# суммирую значения маркеров чтобы получить итоговое значение по которому можно далее произвести сегментацию
# и вывожу лист полученных значений
table_2['Сегмент1']=table_2['Н_РПЛ_П_С1'] + table_2['Н_РПЛ_Т_С1'] + table_2['С_РПЛ_П_С1'] + table_2['С_РПЛ_Т_С1']
list_1 = pd.unique(table_2['Сегмент1'])
sorted(list_1, key=int)

[100.0,
 500.0,
 600.0,
 1000.0,
 1100.0,
 1500.0,
 1600.0,
 2000.0,
 2100.0,
 2500.0,
 2600.0,
 3000.0,
 3100.0,
 3500.0,
 3600.0]

In [13]:
# исходя из понимания смысла заложенного в кодировку маркеров расписысиваю значения сегментов
table_2.loc[table_2['Сегмент1'] == 100, 'Сегмент1_Значение'] = 'Без подписки'
table_2.loc[table_2['Сегмент1'] == 500, 'Сегмент1_Значение'] = 'Без подписки'
table_2.loc[table_2['Сегмент1'] == 600, 'Сегмент1_Значение'] = 'Без подписки'
table_2.loc[table_2['Сегмент1'] == 1000, 'Сегмент1_Значение'] = 'Бывшие подписчики'
table_2.loc[table_2['Сегмент1'] == 1100, 'Сегмент1_Значение'] = 'Бывшие подписчики'
table_2.loc[table_2['Сегмент1'] == 1500, 'Сегмент1_Значение'] = 'Бывшие подписчики'
table_2.loc[table_2['Сегмент1'] == 1600, 'Сегмент1_Значение'] = 'Бывшие подписчики'
table_2.loc[table_2['Сегмент1'] == 2000, 'Сегмент1_Значение'] = 'Новые клиенты. Сразу взяли подписку'
table_2.loc[table_2['Сегмент1'] == 2100, 'Сегмент1_Значение'] = 'Новые подписчики с опытом покупки трансляций'
table_2.loc[table_2['Сегмент1'] == 2500, 'Сегмент1_Значение'] = 'Новые подписчики с опытом покупки трансляций'
table_2.loc[table_2['Сегмент1'] == 2600, 'Сегмент1_Значение'] = 'Новые подписчики с опытом покупки трансляций'
table_2.loc[table_2['Сегмент1'] == 3000, 'Сегмент1_Значение'] = 'Подписчики 2-х сезонов'
table_2.loc[table_2['Сегмент1'] == 3100, 'Сегмент1_Значение'] = 'Подписчики 2-х сезонов'
table_2.loc[table_2['Сегмент1'] == 3500, 'Сегмент1_Значение'] = 'Подписчики 2-х сезонов'
table_2.loc[table_2['Сегмент1'] == 3600, 'Сегмент1_Значение'] = 'Подписчики 2-х сезонов'

In [14]:
# сводная таблица и вывод результатов
v_2 = pd.pivot_table(table_2, values='Marker', index=['Сегмент1_Значение'], aggfunc=np.sum)
v_2

C:\Users\User\Anaconda3\envs\HSE_study\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


Сегмент1_Значение
Без подписки                                    123395
Бывшие подписчики                                28025
Новые клиенты. Сразу взяли подписку               4493
Новые подписчики с опытом покупки трансляций      2124
Подписчики 2-х сезонов                            5951
Name: Marker, dtype: int64

=========================================================================================================================

Вопрос №2. Кто покупает подписки в новом сезоне — новые люди, или у них были и раньше подписки?
А может, они устали покупать трансляции и перешли на подписку?

Краткое резюму по вопросу №2:
 - Судя по данным подписчики распределись в соотношении примерно:
     - 48% это те кто уже имел ранее подписки
     - 35% вновь пришедщие подписчиков которые ранее даже не покупали трансляции.
     - 18% подписчиков купило подписку после того как попробовала трансляции


=========================================================================================================================

# Вопрос №3. Отдельно рассмотрим тех, у кого в прошлом сезоне было куплено более одной подписки. Хочу прокоммуницировать с этими ребятами.

# Что с ними в этом сезоне? Они подписались? Стали покупать разовые? Ничего не купили ещё?

In [15]:
# добавляю колонки с маркерами
table_2.loc[table_2['Новый_сезон_РПЛ']['Подписка'] > 0, 'Н_РПЛ_П_С2'] = 5000
table_2.loc[table_2['Новый_сезон_РПЛ']['Трансляция'] > 0, 'Н_РПЛ_Т_С2'] = 100
table_2.loc[table_2['Старый_сезон_РПЛ']['Подписка'] > 1, 'С_РПЛ_П_С2'] = 10000
table_2= table_2.fillna(0)

In [16]:
# суммирую значения маркеров чтобы получить итоговое значение по которому можно далее произвести сегментацию
# и вывожу лист полученных значений
table_2['Сегмент2']=table_2['Н_РПЛ_П_С2'] + table_2['Н_РПЛ_Т_С2'] + table_2['С_РПЛ_П_С2']
list_2 = pd.unique(table_2['Сегмент2'])
sorted(list_2, key=int)

[0.0, 100.0, 5000.0, 5100.0, 10000.0, 10100.0, 15000.0, 15100.0]

In [17]:
# исходя из понимания смысла заложенного в кодировку маркеров расписысиваю значения сегментов
table_2.loc[table_2['Сегмент2'] == 0, 'Сегмент2_Значение'] = '-'
table_2.loc[table_2['Сегмент2'] == 100, 'Сегмент2_Значение'] = '-'
table_2.loc[table_2['Сегмент2'] == 5000, 'Сегмент2_Значение'] = '-'
table_2.loc[table_2['Сегмент2'] == 5100, 'Сегмент2_Значение'] = '-'
table_2.loc[table_2['Сегмент2'] == 10000, 'Сегмент2_Значение'] = 'Еще ничего не купили'
table_2.loc[table_2['Сегмент2'] == 10100, 'Сегмент2_Значение'] = 'Стали покупать разовые'
table_2.loc[table_2['Сегмент2'] == 15000, 'Сегмент2_Значение'] = 'Оформили подписку в новом сезоне'
table_2.loc[table_2['Сегмент2'] == 15100, 'Сегмент2_Значение'] = 'Оформили подписку в новом сезоне + покупка разовых трансляций'

In [18]:
# сводная таблица и вывод результатов
v_3 = pd.pivot_table(table_2, values='Marker', index=['Сегмент2_Значение'], aggfunc=np.sum)
v_3

Сегмент2_Значение
-                                                                151693
Еще ничего не купили                                               7534
Оформили подписку в новом сезоне                                   3885
Оформили подписку в новом сезоне + покупка разовых трансляций       149
Стали покупать разовые                                              727
Name: Marker, dtype: int64

=========================================================================================================================

Вопрос №3. Отдельно рассмотрим тех, у кого в прошлом сезоне было куплено более одной подписки. Хочу прокоммуницировать с этими ребятами.
Что с ними в этом сезоне? Они подписались? Стали покупать разовые? Ничего не купили ещё??

Краткое резюму по вопросу №3:
 - Данные показывают что из тех кто в прошлом сезоне покупал более 1 подписки демонстрируют слабую активность, а именно:
     - 61% еще не сделали ни одной покупки (ни подписки, ни трансляции)
     - 6% стали покупать разовые трансляции
 - И только 33% проявляют положительную активность:
     - 32% продлил подписку
     - 1% продлил подписку и помимо этого покупает трансляции


=========================================================================================================================

# Вопрос №4. Очень много покупок трансляций. Они впервые что-то покупают/иногда у нас покупали трансляции/раньше они были с подписками?

In [19]:
# добавляю колонки с маркерами
table_2.loc[table_2['Новый_сезон_РПЛ']['Подписка'] > 0, 'Н_РПЛ_П_С3'] = 200
table_2.loc[table_2['Новый_сезон_РПЛ']['Трансляция'] > 1, 'Н_РПЛ_Т_С3'] = 2000
table_2.loc[table_2['Новый_сезон_РПЛ']['Трансляция'] == 1, 'Н_РПЛ_Т_С3'] = 1000
table_2.loc[table_2['Старый_сезон_РПЛ']['Подписка'] > 0, 'С_РПЛ_П_С3'] = 100 
table_2.loc[table_2['Старый_сезон_РПЛ']['Трансляция'] > 0, 'С_РПЛ_Т_С3'] = 50
table_2= table_2.fillna(0)

In [20]:
# суммирую значения маркеров чтобы получить итоговое значение по которому можно далее произвести сегментацию
# и вывожу лист полученных значений
table_2['Сегмент3']=table_2['Н_РПЛ_П_С3'] + table_2['Н_РПЛ_Т_С3'] + table_2['С_РПЛ_П_С3'] +  table_2['С_РПЛ_Т_С3']
list_3 = pd.unique(table_2['Сегмент3'])
sorted(list_3, key=int)

[50.0,
 100.0,
 150.0,
 200.0,
 250.0,
 300.0,
 350.0,
 1000.0,
 1050.0,
 1100.0,
 1150.0,
 1200.0,
 1250.0,
 1300.0,
 1350.0,
 2000.0,
 2050.0,
 2100.0,
 2150.0,
 2200.0,
 2250.0,
 2300.0,
 2350.0]

In [21]:
# исходя из понимания смысла заложенного в кодировку маркеров расписысиваю значения сегментов
table_2.loc[table_2['Сегмент3'] == 50, 'Сегмент3_Значение'] = '01.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 0, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 100, 'Сегмент3_Значение'] = '02.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 0, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 150, 'Сегмент3_Значение'] = '03.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 0, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 200, 'Сегмент3_Значение'] = '04.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 0, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 250, 'Сегмент3_Значение'] = '05.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 0, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 300, 'Сегмент3_Значение'] = '06.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 0, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 350, 'Сегмент3_Значение'] = '07.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 0, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 1000, 'Сегмент3_Значение'] = '08.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 1050, 'Сегмент3_Значение'] = '09.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 1100, 'Сегмент3_Значение'] = '10.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 1150, 'Сегмент3_Значение'] = '11.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 1200, 'Сегмент3_Значение'] = '12.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 1250, 'Сегмент3_Значение'] = '13.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 1300, 'Сегмент3_Значение'] = '14.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 1350, 'Сегмент3_Значение'] = '15.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 2000, 'Сегмент3_Значение'] = '16.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС > 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 2050, 'Сегмент3_Значение'] = '17.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС > 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 2100, 'Сегмент3_Значение'] = '18.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС > 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 2150, 'Сегмент3_Значение'] = '19.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС > 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 2200, 'Сегмент3_Значение'] = '20.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС > 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 2250, 'Сегмент3_Значение'] = '21.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС > 1, ПС - не покупал'
table_2.loc[table_2['Сегмент3'] == 2300, 'Сегмент3_Значение'] = '22.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС > 1, ПС - покупал'
table_2.loc[table_2['Сегмент3'] == 2350, 'Сегмент3_Значение'] = '23.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС > 1, ПС - покупал'

In [22]:
# сводная таблица и вывод результатов
v_4 = pd.pivot_table(table_2, values='Marker', index=['Сегмент3_Значение'], aggfunc=np.sum)
v_4

Сегмент3_Значение
01.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 0, ПС - покупал       103128
02.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 0, ПС - не покупал     19935
03.Подписка: НС - не активна, ПС - была активна     ||  Трансляции: НС = 0, ПС - покупал         6320
04.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 0, ПС - не покупал      4493
05.Подписка: НС - активна,    ПС - не была активна  ||  Трансляции: НС = 0, ПС - покупал         1597
06.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 0, ПС - не покупал      3701
07.Подписка: НС - активна,    ПС - была активна     ||  Трансляции: НС = 0, ПС - покупал         1986
08.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 1, ПС - не покупал      7737
09.Подписка: НС - не активна, ПС - не была активна  ||  Трансляции: НС = 1, ПС - покупал         8213
10.Подписка: НС - не активна, ПС - была активна     ||  Трансляц

=========================================================================================================================

Вопрос №4. Очень много покупок трансляций. Они впервые что-то покупают/иногда у нас покупали трансляции/раньше они были с подписками?

Краткое резюму по вопросу №4:
 - для анализа ситуации сделал довольно подробную сегментацию клиентов в которой отобразил статус подписки в текущем / прошлом сезоне и разделил на три категории по кол-ву трансляций купленных в текущем сезоне (>1, =1, =0)
 - Далее анализируя получившуюся картину видно что из общего числа уникальных клиентов около 163 000 тех кто покупал трансляции в прошлом сезоне, но ничего не покупал в текущем и никогда не брал подписки около 103 000 (63%). Получается что больше половоны клиентов не являются постоянными или же в текущем 2019 году не было значимых событий в РПЛ чтобы привлечь людей к покупкам.
 - Так же стоит выделить второй по величине пласт клиентов у которых в прошлом сезона была активна подписка, но которые не покупают трансляции и в текущем сезоне они ее не продлили (около 20 000 (12%))
 - Если говорить про трансляции то из тех кто приобретал более 1 трансляции в текщем сезоне это клиенты которые преимущественно без подписки, либо были подписчиками в прошлом сезоне. Общее кол-во таких клиентов составило около 5000(3%)
 - Тех же кто покупал всего 1 трансляцию в текущем сезоне составило около 18 000 (11%). Эти клиенты в основной своей массе, так же как и сегмент клиентов из предыдуей категории не являлись подписчиками. 


=========================================================================================================================

# Вопрос №5. Может, уже есть те, кто покупал трансляции 2 и более раз? Им выгоднее подписка. Тоже думаем сделать под них специальный оффер и прокоммуницировать. Сколько таких?

In [23]:
# добавляю колонки с маркерами
table_2.loc[table_2['Новый_сезон_РПЛ']['Подписка'] > 0, 'Н_РПЛ_П_С4'] = -2
table_2.loc[table_2['Новый_сезон_РПЛ']['Трансляция'] > 1, 'Н_РПЛ_Т_С4'] = 1
table_2.loc[table_2['Старый_сезон_РПЛ']['Подписка'] > 0, 'С_РПЛ_П_С4'] = -2 
table_2.loc[table_2['Старый_сезон_РПЛ']['Трансляция'] > 1, 'С_РПЛ_Т_С4'] = 1
table_2= table_2.fillna(0)

In [24]:
# суммирую значения маркеров чтобы получить итоговое значение по которому можно далее произвести сегментацию
# и вывожу лист полученных значений
table_2['Сегмент4']=table_2['Н_РПЛ_П_С4'] + table_2['Н_РПЛ_Т_С4'] + table_2['С_РПЛ_П_С4'] +  table_2['С_РПЛ_Т_С4']
list_4 = pd.unique(table_2['Сегмент4'])
sorted(list_4, key=int)

[-4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0]

In [25]:
# исходя из понимания смысла заложенного в кодировку маркеров расписысиваю значения сегментов
table_2.loc[table_2['Сегмент4'] > 0, 'Сегмент4_Значение'] = 'Больше двух трансляций без подписки'
table_2.loc[table_2['Сегмент4'] < 1, 'Сегмент4_Значение'] = 'Прочие'

In [26]:
# сводная таблица и вывод результатов
v_5 = pd.pivot_table(table_2, values='Marker', index=['Сегмент4_Значение'], aggfunc=np.sum)
v_5

Сегмент4_Значение
Больше двух трансляций без подписки     46933
Прочие                                 117055
Name: Marker, dtype: int64

=========================================================================================================================

Вопрос №5. Может, уже есть те, кто покупал трансляции 2 и более раз? Им выгоднее подписка. Тоже думаем сделать под них специальный оффер и прокоммуницировать. Сколько таких?

Краткое резюму по вопросу №5:
 - Для ответа на этот вопрос взял весь датасет. Посмотрел кто из покупателей брал в хоть в одном сезоне больше 2 трансляций и при этом не брал подписок вообще. Отметил и подсчитал. Получилось 46 933 уникальных клиента которые брали больше 2-х трансляций но не использовали подписки.  


=========================================================================================================================

# Вопрос №6. А матчи с какими командами покупали больше всего?

In [27]:
# сделал выборку из исходной таблицы нужного столбца
ct1=df['title']
ct1.head()

0      Зенит – Спартак. Российская Премьер-лига. 6 тур
1       Ростов - Рубин. Российская Премьер-лига. 6 тур
2      Зенит – Спартак. Российская Премьер-лига. 6 тур
3          ЦСКА - Урал. Российская Премьер-лига. 6 тур
4    Краснодар - Локомотив. Российская Премьер-лига...
Name: title, dtype: object

In [28]:
# почистил список удалив значеия из ряда строк чтобы в начале строки была команда.
ct2=ct1

ct2 = pd.Series(ct2).str.replace('Лига наций. ', '', regex=True)
ct2 = pd.Series(ct2).str.replace('Лига Европы. 3-й отборочный раунд. ', '', regex=True)

# отбросил ненужный текст после названия команд.
ct3 = ct2.str.split('.').str.get(0)

# привел к единому значению значения между названиями команд
ct3 = pd.Series(ct3).str.replace(' – ', '!', regex=True)
ct3 = pd.Series(ct3).str.replace(' - ', '!', regex=True)

# сформировал два списка команд (те названия что до разделителя и те что после)
ct4_1 = ct3.str.split('!').str.get(0)
ct4_2 = ct3.str.split('!').str.get(1)

# склеил список и преобразовал в датафрейм
ct5 = pd.concat([ct4_1, ct4_2])

ct6 = pd.DataFrame(ct5)

# добавил техническую колонку для удобства подсчета кол-ва значений
ct6['Marker'] = 1

# вывод результата
v_6 = pd.pivot_table(ct6, values='Marker', index=['title'], aggfunc=np.sum)
v_6.sort_values(by='Marker', ascending=False)

,Marker
title,
Спартак,156032
Зенит,96004
ЦСКА,86697
Прямой эфир канала МАТЧ ПРЕМЬЕР,73161
Ростов,38671
Локомотив,36126
Динамо,32022
Краснодар,31393
Рубин,24073


=========================================================================================================================

Вопрос №6. А матчи с какими командами покупали больше всего?

Краткое резюму по вопросу №6:
 - ТОП 3 (отрыв тройки лидеров от остальных команд практически в 3 и более раз):
     - Спартак
     - Зенит
     - ЦСКА


=========================================================================================================================

# Резюме для заказчика

    Анализ кол-ва продаж и уникальных заказчиков показал что Январь, Февраль, Июнь являються провальными с точки зрения продаж. Возможно чтобы "подхлеснуть" уровень продаж в указаные месяцы стоит использовать больее агрессивную систему скидок, либо сформировать предложения которе будут состоять из нескольких месяцев (так же включая в себя провальные) При этом при пересчете стоисмости на 1 месяц клиент мог бы увидеть что следующие за провальным месяцем интересные месяца обходились бы ему дешевле но при этом был бы вынужден по такой же "усредненной" цене приобретать и месяцы в которые идет просадка продаж.
 
    Анализ активности покупок в новом сезоне показал что подписчики поделились 50/50 на тех кто продлил подписку и тех кто стал новым подпичсчиком. Не сомнено что приток новых подписок это важный бизнес показатель над которым надо работать и который должен расти, что бизснес развиался. но при этом важно следить за тем чтобы старые подписчики продолжали пользоваться сервисом.
    
    При это анализ тех подписчиков кто купил две и более подписки в прошлом сезоне демонстрирует плохую динамику. Практически 70% из указанной категории подписчиков не продлили свои подписки в новом сезоне.
    
    Так же анализ клиентоской базы в разрезе активности покупок трансляций показывает что более половины клиентской базы это покупатели которые купили трансляцию (одну или несколько в прошлом сезоне), но при этом не проявили никакой актвности в текущем сезоне. Причем указанная группа клиентов ни разу не использовала подписки.
    
    В связи с этим требутеся более детальный анализ клиентской базы и проработка предложений способных не только привлечь новых клиентов но и конвертировать разовых клиентов в постоянных.
    
    Для этого например в рамках указанного датасета был проведен анализ клиентов которые покупали более 2-х трансляций и при этом ни разу не были подписчиками. Пул таких уникальных клинетов составил практически 47 000 клиентов. 
    
    Так же был проведен анализ какие матчи пользовались наибольшим спросом при покупке.
    ТОП 3 (отрыв тройки лидеров от остальных команд практически в 3 и более раз):
     - Спартак
     - Зенит
     - ЦСКА

=========================================================================================================================